In [1]:
import keras
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
train_df.head()

,PID,Date,Event
0,1028890,201101,2186
1,1028890,201101,7087
2,1028890,201101,4848
3,1028890,201101,2214
4,1028890,201102,7087


In [4]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
lbl.fit(list(train_df['Event'].values)) 
train_df['Event'] = lbl.transform(list(train_df['Event'].values))
print('Shape train: {}\n'.format(train_df.shape))

Shape train: (766787, 3)



In [5]:
train_df.head()

,PID,Date,Event
0,1028890,201101,657
1,1028890,201101,3763
2,1028890,201101,2671
3,1028890,201101,678
4,1028890,201102,3763


In [6]:
dataset=train_df.values
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

(513747, 253040)


In [7]:
train

array([[1028890,  201101,     657],
       [1028890,  201101,    3763],
       [1028890,  201101,    2671],
       ..., 
       [1016736,  201104,    1702],
       [1016736,  201104,    1118],
       [1016736,  201105,    1702]])

In [8]:
test

array([[1016736,  201105,    1118],
       [1016736,  201105,    1054],
       [1016736,  201106,    1568],
       ..., 
       [1022024,  201308,    5078],
       [1022024,  201308,    5095],
       [1022024,  201308,    5072]])

In [9]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 2]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 2])
    return np.array(dataX), np.array(dataY)

In [10]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [11]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [12]:
trainX

array([[[ 657]],

       [[3763]],

       [[2671]],

       ..., 
       [[4048]],

       [[2190]],

       [[1702]]])

In [13]:
trainY

array([3763, 2671,  678, ..., 2190, 1702, 1118])

In [14]:
testX

array([[[1118]],

       [[1054]],

       [[1568]],

       ..., 
       [[5089]],

       [[5058]],

       [[5078]]])

In [15]:
testY

array([1054, 1568, 1054, ..., 5058, 5078, 5095])

In [16]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Embedding
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# create and fit the LSTM network
model = Sequential()
#model.add(Embedding(20000,128))
model.add(LSTM(6000, input_shape=(1, look_back),return_sequences=True))
model.add(Dense(1000,activation='relu'))
model.add(LSTM(600))
model.add(Dense(100,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['categorical_accuracy'])
model.fit(trainX, trainY, epochs=2, batch_size=1000, verbose=2)

In [ ]:
score, acc = model.evaluate(trainX, trainY,batch_size=100)
print('Test score:', score)
print('Test accuracy:', acc)